# Analyzing Tabular Data with pyspark.sql

Tabular data refers to a two-dimensional table like excel format. In this notebook, I'll cover data manipulation with `pyspark.sql`.

## Creating SparkSession

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.getOrCreate()

## Reading Data

PySpark uses the SparkReader object to directly read any kind of data in a DataFrame. Let's read a CSV file with the `spark.read.csv` method. Note that PySpark can infer the schema of a CSV file by setting the `inferSchema` optional parameter to True. For the `sep` paramter you can set the sign between values of your dataset. In our case, the dataset contains the `|` sign.

In [2]:
import os
DIRECTORY = "./data/broadcast_logs" 
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"), 
    sep="|", 
    header=True, 
    inferSchema=True, 
    timestampFormat="yyyy-MM-dd", 
)

In [3]:
logs.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string 

## Selecting Columns

The `select` method can take one or more column objects and return a DataFrame containing only the listed columns. Let's take only three columns without truncating:

In [4]:
logs.select("BroadcastLogID", "LogServiceID", "LogDate").show(5, False)

+--------------+------------+-------------------+
|BroadcastLogID|LogServiceID|LogDate            |
+--------------+------------+-------------------+
|1196192316    |3157        |2018-08-01 00:00:00|
|1196192317    |3157        |2018-08-01 00:00:00|
|1196192318    |3157        |2018-08-01 00:00:00|
|1196192319    |3157        |2018-08-01 00:00:00|
|1196192320    |3157        |2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 5 rows



You can use four different ways to get columns with the `select` as the following methods:

In [5]:
# 1. Using the string to column conversion
logs.select("BroadCastLogID", "LogServiceID", "LogDate").show(3)

+--------------+------------+-------------------+
|BroadCastLogID|LogServiceID|            LogDate|
+--------------+------------+-------------------+
|    1196192316|        3157|2018-08-01 00:00:00|
|    1196192317|        3157|2018-08-01 00:00:00|
|    1196192318|        3157|2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 3 rows



In [6]:
# 2. Using the string with the * sign
logs.select(*["BroadCastLogID", "LogServiceID", "LogDate"]).show(3)

+--------------+------------+-------------------+
|BroadCastLogID|LogServiceID|            LogDate|
+--------------+------------+-------------------+
|    1196192316|        3157|2018-08-01 00:00:00|
|    1196192317|        3157|2018-08-01 00:00:00|
|    1196192318|        3157|2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 3 rows



In [7]:
# 3. Passing the column object explicitly
logs.select(F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate")).show(3)         

+--------------+------------+-------------------+
|BroadCastLogID|LogServiceID|            LogDate|
+--------------+------------+-------------------+
|    1196192316|        3157|2018-08-01 00:00:00|
|    1196192317|        3157|2018-08-01 00:00:00|
|    1196192318|        3157|2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 3 rows



In [8]:
# 4. Passing the column object with the * sign
logs.select(*[F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate")]).show(3)

+--------------+------------+-------------------+
|BroadCastLogID|LogServiceID|            LogDate|
+--------------+------------+-------------------+
|    1196192316|        3157|2018-08-01 00:00:00|
|    1196192317|        3157|2018-08-01 00:00:00|
|    1196192318|        3157|2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 3 rows



You can also take the columns with chunks using `numpy`. 

In [9]:
import numpy as np
column_split = np.array_split(np.array(logs.columns), len(logs.columns) // 3) 
print(column_split)

[array(['BroadcastLogID', 'LogServiceID', 'LogDate'], dtype='<U22'), array(['SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID'],
      dtype='<U22'), array(['CategoryID', 'ClosedCaptionID', 'CountryOfOriginID'], dtype='<U22'), array(['DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID'],
      dtype='<U22'), array(['ProgramClassID', 'FilmClassificationID', 'ExhibitionID'],
      dtype='<U22'), array(['Duration', 'EndTime', 'LogEntryDate'], dtype='<U22'), array(['ProductionNO', 'ProgramTitle', 'StartTime'], dtype='<U22'), array(['Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID'],
      dtype='<U22'), array(['BroadcastOriginPointID', 'CompositionID', 'Producer1'],
      dtype='<U22'), array(['Producer2', 'Language1', 'Language2'], dtype='<U22')]


Let's take a look at the first four chunks.

In [10]:
for x in column_split[:4]:
    logs.select(*x).show(3, False)

+--------------+------------+-------------------+
|BroadcastLogID|LogServiceID|LogDate            |
+--------------+------------+-------------------+
|1196192316    |3157        |2018-08-01 00:00:00|
|1196192317    |3157        |2018-08-01 00:00:00|
|1196192318    |3157        |2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 3 rows

+----------+-------------------+----------------------+
|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|
+----------+-------------------+----------------------+
|1         |4                  |null                  |
|2         |null               |null                  |
|3         |null               |null                  |
+----------+-------------------+----------------------+
only showing top 3 rows

+----------+---------------+-----------------+
|CategoryID|ClosedCaptionID|CountryOfOriginID|
+----------+---------------+-----------------+
|13        |3              |3                |
|null      |1       

## Deleting Columns

You can delete the columns you want with the `drop`method. Let's delete the BroadcastLogID, SequenceNO columns:

In [11]:
logs = logs.drop("BroadcastLogID", "SequenceNO")

Let's check them out.

In [12]:
print("BroadcastLogID" in logs.columns)
print("SequenceNo" in logs.columns)

False
False


You can take the columns you want to select using a `for` loop.

In [13]:
logs = logs.select(*[x for x in logs.columns if x not in ["BroadcastLogID", "SequenceNO"]])

In [14]:
len(logs.columns)

28

## Creating New Columns

There are two ways to create new columns: using the `select` and `withColumn` methods. To show these, let's handle the duration column.

In [15]:
logs.select(F.col("Duration")).show(5)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows



Let's take a look at its type.

In [16]:
logs.select(F.col("Duration")).dtypes

[('Duration', 'string')]

With the `substr` method, let's extract a substring from this column. The `substr` method takes two parameters. You can pass the position of where the sub-string starts in first parameter and the length of the sub-string in second parameter.

In [17]:
logs.select(
    F.col("Duration"), 
    # Let's use the cast method to convert to Integer.
    F.col("Duration").substr(1, 2).cast("int").alias("dur_hours"), 
    F.col("Duration").substr(4, 2).cast("int").alias("dur_minutes"), 
    F.col("Duration").substr(7, 2).cast("int").alias("dur_seconds"), 
    # Let's use the distinct method to avoid seeing identical rows.
).distinct().show(3)

+----------------+---------+-----------+-----------+
|        Duration|dur_hours|dur_minutes|dur_seconds|
+----------------+---------+-----------+-----------+
|00:04:52.0000000|        0|          4|         52|
|00:10:06.0000000|        0|         10|          6|
|00:26:41.0000000|        0|         26|         41|
+----------------+---------+-----------+-----------+
only showing top 3 rows



Let’s set all these values into a single second field. To do this, let me convert hour and minute into second.

In [18]:
logs.select(
    F.col("Duration"),
    (F.col("Duration").substr(1, 2).cast("int") * 60 * 60 
     + F.col("Duration").substr(4, 2).cast("int") * 60
     + F.col("Duration").substr(7, 2).cast("int")) \
    .alias("Duration_seconds"),) \
    .distinct().show(5)

+----------------+----------------+
|        Duration|Duration_seconds|
+----------------+----------------+
|01:59:30.0000000|            7170|
|00:31:00.0000000|            1860|
|00:28:08.0000000|            1688|
|00:10:30.0000000|             630|
|06:00:00.0000000|           21600|
+----------------+----------------+
only showing top 5 rows



You can also add a column at the end of a DataFrame with the `withColumn` method. Let me show you.

In [19]:
logs = logs.withColumn("Duration_seconds",
                       (F.col("Duration").substr(1, 2).cast("int") * 60 * 60
                        + F.col("Duration").substr(4, 2).cast("int") * 60
                        + F.col("Duration").substr(7, 2).cast("int")),)
logs.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttenti

## Renaming Columns

As mentioned, you can rename the columns with the `select` and `alias` methods. To do this, you can use the `withColumnRenamed` method. Let's use this method to convert the name of a column into lower case.

In [20]:
logs = logs.withColumnRenamed("Duration_seconds", "duration_seconds")

You can want to rename all the columns of your DataFrame with the `toDF` method. Let me show you.

In [21]:
logs.toDF(*[x.lower() for x in logs.columns]).printSchema()

root
 |-- logserviceid: integer (nullable = true)
 |-- logdate: timestamp (nullable = true)
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- productionsourceid: integer (nullable = true)
 |-- programclassid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- logentrydate: timestamp (nullable = true)
 |-- productionno: string (nullable = true)
 |-- programtitle: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- subtitle: string (nullable = true)
 |-- networkaffiliationid: integer (nullable = true)
 |-- specialattenti

##  Reordering Columns

To sort the columns alphabetically, you can use the `sort` method.

In [22]:
logs.select(sorted(logs.columns)).printSchema()

root
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- BroadcastOriginPointID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CompositionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- Language1: integer (nullable = true)
 |-- Language2: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- Producer1: string (nullable = true)
 |-- Producer2: string (nullable = true)
 |-- ProductionNO: 

## Diagnosing a DataFrame with the Describe & Summary Methods 

To take a look at summary statistics (count, mean, standard deviation, min, and max) on all numerical and string columns, you can use the `describe` method. Let's look at the summary of first three columns with a for loop.

In [23]:
for i in logs.columns[:3]:
    logs.describe(i).show()

+-------+------------------+
|summary|      LogServiceID|
+-------+------------------+
|  count|            238945|
|   mean| 3450.890284375065|
| stddev|199.50673962554765|
|    min|              3157|
|    max|              3925|
+-------+------------------+

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+

+-------+-------------------+
|summary|AudienceTargetAgeID|
+-------+-------------------+
|  count|              16112|
|   mean| 3.4929245283018866|
| stddev| 1.0415963394745125|
|    min|                  1|
|    max|                  4|
+-------+-------------------+



The `describe` has a fixed set of metrics, while the `summary` will take functions as parameters and apply them to all columns. You can also customize the statistics you want to see with the `summary` method. By default, the `summary` method shows all statistics like the `describe` method, adding the approximate 25-50% and 75% percentiles. However, you can specify statistisc you want to see. Let me show you. 

In [24]:
for i in logs.columns[:3]:
    logs.select(i).summary().show() 

+-------+------------------+
|summary|      LogServiceID|
+-------+------------------+
|  count|            238945|
|   mean| 3450.890284375065|
| stddev|199.50673962554765|
|    min|              3157|
|    25%|              3287|
|    50%|              3379|
|    75%|              3627|
|    max|              3925|
+-------+------------------+

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    25%|
|    50%|
|    75%|
|    max|
+-------+

+-------+-------------------+
|summary|AudienceTargetAgeID|
+-------+-------------------+
|  count|              16112|
|   mean| 3.4929245283018866|
| stddev| 1.0415963394745125|
|    min|                  1|
|    25%|                  4|
|    50%|                  4|
|    75%|                  4|
|    max|                  4|
+-------+-------------------+



Let's limit statistics our want.

In [25]:
for i in logs.columns[:3]: 
    logs.select(i).summary("min", "20%", "80%", "max").show() 

+-------+------------+
|summary|LogServiceID|
+-------+------------+
|    min|        3157|
|    20%|        3269|
|    80%|        3657|
|    max|        3925|
+-------+------------+

+-------+
|summary|
+-------+
|    min|
|    20%|
|    80%|
|    max|
+-------+

+-------+-------------------+
|summary|AudienceTargetAgeID|
+-------+-------------------+
|    min|                  1|
|    20%|                  3|
|    80%|                  4|
|    max|                  4|
+-------+-------------------+



Thanks for reading. I hope you enjoy it 😀

Don't forget to follow us on [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎